# Création du graph

In [1]:
import networkx as nx
import pandas as pd
import math
import json

***Import des données***

In [2]:
col_1 = [["point_1", "point_1", "point_1", "point_2", "point_2", "point_2", "duree (min)", "distance (km)", 
          "empreinte carbone (gCO2)", "prix (euros)"],
        ["nom", "latitude", "longitude", "nom", "latitude", "longitude", "", "", "", ""]]
col_2 = [["point_1", "point_1", "point_1", "point_2", "point_2", "point_2", "duree (min)", "distance (km)", 
          "empreinte carbone (gCO2)", "prix (euros)"], 
        ["nom", "latitude", "longitude", "nom", "latitude", "longitude", "", "", "", ""]]
col_1 = pd.MultiIndex.from_arrays(col_1)
col_2 = pd.MultiIndex.from_arrays(col_2)

df_avions = pd.read_csv("results/df_avions.csv", sep=",", header=1, names = col_2)
df_bus = pd.read_csv("results/df_bus.csv", sep=",", header=1, names = col_1)
df_intercites = pd.read_csv("results/df_intercites.csv", sep=",", header=1, names = col_1)
df_ter = pd.read_csv("results/df_ter.csv", sep=",", header=1, names = col_1)
df_tgv = pd.read_csv("results/df_tgv.csv", sep=",", header=1, names = col_1)

***Fonctions***

In [3]:
def graph(df: object, transport: str) -> object: # qd avions a les empreintes carbones on fait put weigth direct in
    """
    Retourne le graph définie par le dataframe df ou :
     -> Les noeuds sont définis "nom"
        Les attribus du noeud sont "latitude" et "longitude"
     -> Les arretes sont définis ("nom", "point1") et ("point2", "nom")
        Les attribus de l'arrete sont "duree (min)", "distance (km)", "empreinte carbone (gCO2)" et "prix (euros)"
    """
    G = nx.Graph()
    dic_point_pos = {}
    for point1, lat1, long1, point2, lat2, long2 in df[[("point_1", "nom"), ("point_1", "latitude"), 
                                                        ("point_1", "longitude"), ("point_2", "nom"), 
                                                        ("point_2", "latitude"), ("point_2", "longitude")]].values:
        dic_point_pos[point1] = (round(lat1, 5), round(long1, 5))
        dic_point_pos[point2] = (round(lat2, 5), round(long2, 5))

    for node, lat_long in dic_point_pos.items():
        G.add_node(node, latitude=lat_long[0], longitude=lat_long[1])

    dic_point1_point2_attrib = {}
    for point1, point2, duree, distance, empreinte_carbone, prix in df[[("point_1", "nom"), ("point_2", "nom"), 
                                                   ("duree (min)", ""), ("distance (km)", ""), 
                                                   ("empreinte carbone (gCO2)", ""), ("prix (euros)", "")]].values:
        dic_point1_point2_attrib[(point1, point2)] = (duree, distance, empreinte_carbone, prix)

    for points, attrib in dic_point1_point2_attrib.items():
        G.add_edge(points[0], points[1], duree=round(attrib[0], 2), distance=round(attrib[1], 2), 
                   empreinte_carbone=round(attrib[2], 2), prix=round(attrib[3], 2), transport=transport)

    return G

def merge_graph(Graphs: list) -> object:
    G = nx.Graph()
    for G_temp in Graphs:
        G = nx.disjoint_union(G, G_temp)
    return G
    
def mins_maxs(edges_data: list) -> list:
    maxs, mins = {}, {}
    for attrib in edges_data:
        for key, value in attrib[2].items():
            if value != str(value):
                if maxs.get(key, 0) < value:
                    maxs[key] = value
                if mins.get(key, 1e12) > value:
                    mins[key] = value
    return [maxs, mins]
        
def ecart(lat_i: float, lat_f: float, long_i: float, long_f: float): #formule de haversine
    r = 6378
    lat_i, long_i = math.radians(lat_i), math.radians(long_i)
    lat_f, long_f = math.radians(lat_f), math.radians(long_f)
    return 2*r*math.asin(math.sqrt(math.sin((lat_i - lat_f)/2)**2 + math.cos(lat_i)*
                                   math.cos(lat_f)*math.sin((long_i-long_f)/2)**2))

def add_connexions(G: object, rayon: int=5) -> object:
    """
    ajoute les connexions avec tous les noeuds à moins de rayon (defaut 5) km du noeud par de la marche
    ASSEZ LONG CAR REGARDE SUR TOUS LES NOEUDS LESQUELS SONT LES PLUS PROCHES
    """
    G_added = G.copy()
    dic_nodes = dict(G_added.nodes.data())
    for node, attribs in dic_nodes.items():
        dict_sorted = {k: v for k, v in sorted(dic_nodes.items(), key=lambda x: ecart(list(x[1].values())[0], 
                                               attribs['latitude'], list(x[1].values())[1], attribs['longitude']))}
        edge_added = 0
        distance = 0
        list_nodes = list(dict_sorted.keys())
        while distance < rayon:
            node_i = list_nodes[edge_added]
            attribs_i = dict_sorted[node_i]
            distance = ecart(attribs['latitude'], attribs_i['latitude'], attribs['longitude'], 
                             attribs_i['longitude'])
            t = 60*distance/3.0
            edge_added += 1
            if node != node_i:
                G_added.add_edge(node, node_i, duree=round(t, 2), distance=round(distance, 3), 
                                 empreinte_carbone=0.0, prix=0.0, transport="marche")
    return G_added

def rel_num_nom(G, list_df):
    """
    relation entre le numéro du noeud et le nom de la position
    on part du principe que pour un nom il n'y a qu'une position et vis versa
    """
    rel = {}
    noms, pos = [], []
    for df in list_df:
        noms += df[("point_1", "nom")].values.tolist() + df[("point_2", "nom")].values.tolist()
        pos_temp = df[[("point_1", "latitude"), ("point_1", "longitude")]].values.tolist() + \
                   df[[("point_2", "latitude"), ("point_2", "longitude")]].values.tolist()
        pos += [(round(pos_i[0], 5), round(pos_i[1], 5)) for pos_i in pos_temp]
    dic_pos_nom = dict(zip(pos, noms))
    for node in G.nodes():
        rel[node] = dic_pos_nom[tuple(G.nodes[node].values())]
    return rel

***Création des graphs / relation entre n° et nom pour le merge de graphs***

In [4]:
G_bus = graph(df_bus, "bus")
#G_avions = graph(df_avions, "avions")
G_tgv = graph(df_tgv, "tgv")
G_intercites = graph(df_intercites, "intercites")
G_ter = graph(df_ter, "ter")

G = merge_graph([G_bus, G_intercites, G_ter, G_tgv]) # METTRE AVIONS ET TGV !!!!!!!!
rayon = 5
G = add_connexions(G, rayon=rayon)

nx.write_gpickle(G, "graph") # data graph définis avec rayon = rayon

In [6]:
rel = rel_num_nom(G, [df_bus, df_intercites, df_ter, df_tgv]) # METTRE AVIONS ET TGV !!!!!!!!
with open('rel', 'w') as out_f:
    json.dump(rel, out_f)